# LCEL
LCEL」(LangChain Expression Language)は、チェーンを簡単に記述するための宣言型の手法

RetrievalQAをそのまま使うとチェーンが組みにくくて使いにくいので今後は基本的にLCELを使っていくことになります

## 下準備

In [3]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter,RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Document load
loader = DirectoryLoader("../datasets/company_documents_dataset_1/", glob="**/*.txt",recursive=True)
raw_docs = loader.load()

# Document split
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    return_each_line=False,
    strip_headers = False 
)
docs = []
for raw_doc in raw_docs:
    source = raw_doc.metadata["source"]
    spilited_docs = markdown_splitter.split_text(raw_doc.page_content)
    for doc in spilited_docs:
        doc.metadata["source"] = source#metadataにsourceを加える
    docs = docs + spilited_docs
markdown_splited_docs = docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300,chunk_overlap=50)
docs = text_splitter.split_documents(docs)

# Embd
vectorstore = Chroma.from_documents(persist_directory="./vecstore/index", documents=docs, embedding=OpenAIEmbeddings())

#llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

# retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# prompt
prompt = PromptTemplate(
    input_variables=["context","question"],
    template="""以下の参考用のテキストの一部を参照して、Questionに回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。
{context}
Question: {question}
Answer: """
)

/Users/rokuroyamaki/anaconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/rokuroyamaki/anaconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecat

## LCEL方式でチェーンを組む

In [4]:
#chain
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {
        "question":RunnablePassthrough(),#invokeで指定したtextが入る。
        "context":retriever 
    }
    |prompt
    |llm
    |StrOutputParser()
  
)
question = "社長の名前は？"
rag_chain.invoke(question)

'漆黒 花太郎'

## ドキュメント検索で使用したコンテキストを確認する

In [5]:
question = "社長の名前は？"
print(retriever.invoke(question))

[Document(page_content='### 社長  \n漆黒 花太郎（しっこく かたろう）', metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '会社概要', 'Header 3': '社長', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'}), Document(page_content='### 社長  \n漆黒 花太郎（しっこく かたろう）', metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '会社概要', 'Header 3': '社長', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'}), Document(page_content='## 社長のプロフィール  \n### 名前  \n漆黒 花太郎（しっこく かたろう）', metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '社長のプロフィール', 'Header 3': '名前', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'}), Document(page_content='## 社長のプロフィール  \n### 名前  \n漆黒 花太郎（しっこく かたろう）', metadata={'Header 1': '株式会社架空ブラック 会社情報', 'Header 2': '社長のプロフィール', 'Header 3': '名前', 'source': '../datasets/company_documents_dataset_1/マニュアル/会社情報.txt'})]


## Chainを可視化する

In [6]:
rag_chain.get_graph().print_ascii()

       +---------------------------------+             
       | Parallel<question,context>Input |             
       +---------------------------------+             
               **               **                     
            ***                   ***                  
          **                         **                
+-------------+              +----------------------+  
| Passthrough |              | VectorStoreRetriever |  
+-------------+              +----------------------+  
               **               **                     
                 ***         ***                       
                    **     **                          
      +----------------------------------+             
      | Parallel<question,context>Output |             
      +----------------------------------+             
                        *                              
                        *                              
                        *                       